<h2 align =\\\center\\\> Movie Recommendation Engine(Content Based Filtering)

## Dataset Information

   Data on Movies from IMDB (Includes Some Television as Well). Movie IDs to help gather much of this data come from one or two Kaggle projects. There is a workflow from original cobbled together spreadsheets to the final product with 27 variables and over 5000 observations. \
   Content based Filtering

## Import Modules And Load Dataset

In [2]:

import pandas as pd 
import numpy as np 
import re
import nltk
pd.set_option('display.max_columns', None)


In [12]:
df = pd.read_csv('dataset.csv')
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [13]:
df['Plot'][0]

'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'

## Data Preprocessing

In [14]:
#convert lowercase, remove nums, punctuations and spaces
df['cleaned_plot'] = df['Plot'].str.lower()
df['cleaned_plot'] = df['cleaned_plot'].apply(lambda x: re.sub('[^a-zA-z]', ' ', x))
df['cleaned_plot'] = df['cleaned_plot'].apply(lambda x: re.sub('\s+', ' ', x))
df['cleaned_plot']

0      two imprisoned men bond over a number of years...
1      the aging patriarch of an organized crime dyna...
2      the early life and career of vito corleone in ...
3      when the menace known as the joker emerges fro...
4      a jury holdout attempts to prevent a miscarria...
                             ...                        
245    the desperate life of a chronic alcoholic is f...
246    a something supervising staff member of a resi...
247    a newspaper editor uses every trick in the boo...
248    an old man makes a long journey by lawn mover ...
249    a mumbai teen reflects on his upbringing in th...
Name: cleaned_plot, Length: 250, dtype: object

In [15]:
# tokenization
df['cleaned_plot'] = df['cleaned_plot'].apply(lambda x: nltk.word_tokenize(x))
df['cleaned_plot']


0      [two, imprisoned, men, bond, over, a, number, ...
1      [the, aging, patriarch, of, an, organized, cri...
2      [the, early, life, and, career, of, vito, corl...
3      [when, the, menace, known, as, the, joker, eme...
4      [a, jury, holdout, attempts, to, prevent, a, m...
                             ...                        
245    [the, desperate, life, of, a, chronic, alcohol...
246    [a, something, supervising, staff, member, of,...
247    [a, newspaper, editor, uses, every, trick, in,...
248    [an, old, man, makes, a, long, journey, by, la...
249    [a, mumbai, teen, reflects, on, his, upbringin...
Name: cleaned_plot, Length: 250, dtype: object

In [16]:
# remove duplicate words
duplicates = nltk.corpus.stopwords.words('english')
plot = []
for sen in df['cleaned_plot']:
    temp = []
    for word in sen:
        if word not in duplicates and len(word) > 3:
            temp.append(word)

    plot.append(temp)
plot


[['imprisoned',
  'bond',
  'number',
  'years',
  'finding',
  'solace',
  'eventual',
  'redemption',
  'acts',
  'common',
  'decency'],
 ['aging',
  'patriarch',
  'organized',
  'crime',
  'dynasty',
  'transfers',
  'control',
  'clandestine',
  'empire',
  'reluctant'],
 ['early',
  'life',
  'career',
  'vito',
  'corleone',
  'york',
  'portrayed',
  'michael',
  'expands',
  'tightens',
  'grip',
  'family',
  'crime',
  'syndicate'],
 ['menace',
  'known',
  'joker',
  'emerges',
  'mysterious',
  'past',
  'wreaks',
  'havoc',
  'chaos',
  'people',
  'gotham',
  'dark',
  'knight',
  'must',
  'accept',
  'greatest',
  'psychological',
  'physical',
  'tests',
  'ability',
  'fight',
  'injustice'],
 ['jury',
  'holdout',
  'attempts',
  'prevent',
  'miscarriage',
  'justice',
  'forcing',
  'colleagues',
  'reconsider',
  'evidence'],
 ['german',
  'occupied',
  'poland',
  'world',
  'oskar',
  'schindler',
  'gradually',
  'becomes',
  'concerned',
  'jewish',
  'workf

In [17]:
df['cleaned_plot'] = plot

In [18]:
df['cleaned_plot']

0      [imprisoned, bond, number, years, finding, sol...
1      [aging, patriarch, organized, crime, dynasty, ...
2      [early, life, career, vito, corleone, york, po...
3      [menace, known, joker, emerges, mysterious, pa...
4      [jury, holdout, attempts, prevent, miscarriage...
                             ...                        
245    [desperate, life, chronic, alcoholic, followed...
246    [something, supervising, staff, member, reside...
247    [newspaper, editor, uses, every, trick, book, ...
248    [makes, long, journey, lawn, mover, tractor, m...
249    [mumbai, teen, reflects, upbringing, slums, ac...
Name: cleaned_plot, Length: 250, dtype: object

In [19]:
df['Genre'] = df['Genre'].apply(lambda x: x.split(','))
df['Actors'] = df['Actors'].apply(lambda x: x.split(',')[:4])
df['Director'] = df['Director'].apply(lambda x: x.split(','))


In [23]:
def clean(x):
    temp = []
    for word in x:
        temp.append(word.lower().replace(' ', ''))
    return temp 


In [24]:
df['Genre'] = [clean(x) for x in df['Genre']]
df['Actor'] = [clean(x) for x in df['Actors']]
df['Director'] = [clean(x) for x in df['Director']]


In [27]:
columns = ['cleaned_plot', 'Genre', 'Actors', 'Director']
l = []
for i in range(len(df)):
    word = ''
    for col in columns:
        word += ' '.join(df[col][i])+' '
    l.append(word)
l

['imprisoned bond number years finding solace eventual redemption acts common decency crime drama Tim Robbins  Morgan Freeman  Bob Gunton  William Sadler frankdarabont ',
 'aging patriarch organized crime dynasty transfers control clandestine empire reluctant crime drama Marlon Brando  Al Pacino  James Caan  Richard S. Castellano francisfordcoppola ',
 'early life career vito corleone york portrayed michael expands tightens grip family crime syndicate crime drama Al Pacino  Robert Duvall  Diane Keaton  Robert De Niro francisfordcoppola ',
 'menace known joker emerges mysterious past wreaks havoc chaos people gotham dark knight must accept greatest psychological physical tests ability fight injustice action crime drama Christian Bale  Heath Ledger  Aaron Eckhart  Michael Caine christophernolan ',
 'jury holdout attempts prevent miscarriage justice forcing colleagues reconsider evidence crime drama Martin Balsam  John Fiedler  Lee J. Cobb  E.G. Marshall sidneylumet ',
 'german occupied p

In [30]:
#combining data with title 
df['cleanip'] = l
df = df[['Title', 'cleanip']]
df.head()

,Title,cleanip
0,The Shawshank Redemption,imprisoned bond number years finding solace ev...
1,The Godfather,aging patriarch organized crime dynasty transf...
2,The Godfather: Part II,early life career vito corleone york portrayed...
3,The Dark Knight,menace known joker emerges mysterious past wre...
4,12 Angry Men,jury holdout attempts prevent miscarriage just...


## Feature Extraction

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf = TfidfVectorizer()
features = tfidf.fit_transform(df['cleanip'])

In [34]:
#create cos similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
cossim = cosine_similarity(features)
print(cossim)

[[1.         0.02532959 0.02358074 ... 0.00338141 0.00331289 0.00301446]
 [0.02532959 1.         0.17787708 ... 0.00358586 0.0035132  0.00319672]
 [0.02358074 0.17787708 1.         ... 0.00333828 0.00327064 0.00297601]
 ...
 [0.00338141 0.00358586 0.00333828 ... 1.         0.00340432 0.00309765]
 [0.00331289 0.0035132  0.00327064 ... 0.00340432 1.         0.00303488]
 [0.00301446 0.00319672 0.00297601 ... 0.00309765 0.00303488 1.        ]]


## Movie Recommendation

In [39]:
idx = pd.Series(df['Title'])
idx.head()

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [69]:
def rec_movies(title):
    movies = []
    indx = idx[idx==title].index[0] 
    #print(indx)
    score = pd.Series(cossim[indx]).sort_values(ascending=False)
    top10 = list(score.iloc[1:11].index)#ignore 0th index as that will be the same movie
    #print(top10)


    for i in top10:
        movies.append(df['Title'][i])
    return movies

In [70]:
rec_movies('Inception')


['The Dark Knight Rises',
 'The Revenant',
 'Interstellar',
 'Mad Max: Fury Road',
 'Blood Diamond',
 'Catch Me If You Can',
 'Shutter Island',
 'Spider-Man: Homecoming',
 'The Departed',
 'The Prestige']

In [66]:
rec_movies('The Dark Knight Rises')

['The Dark Knight',
 'Inception',
 'Batman Begins',
 'The Prestige',
 'The Lord of the Rings: The Fellowship of the Ring',
 'Die Hard',
 'Léon: The Professional',
 'Mad Max: Fury Road',
 'Spider-Man: Homecoming',
 'Sin City']

In [71]:
rec_movies('Die Hard')


['The Sixth Sense',
 'The Great Dictator',
 'The Great Escape',
 'The Dark Knight Rises',
 'Twelve Monkeys',
 'The Nightmare Before Christmas',
 'Rope',
 'The Hustler',
 'Dog Day Afternoon',
 'Django Unchained']

In [73]:
rec_movies('The Avengers')


['Guardians of the Galaxy Vol. 2',
 'Guardians of the Galaxy',
 'Spider-Man: Homecoming',
 'Rush',
 'Aliens',
 'The Martian',
 'Interstellar',
 'Blade Runner',
 'Kill Bill: Vol. 1',
 'The Thing']